In [ ]:
# Use matplotlib interactively inside jupyter notebooks
%matplotlib notebook

In [ ]:
from scipy.special import ellipj, ellipk

# Define Jacobi elliptic functions parametrized with k, k^2 = m.
def sn(x, k):
    result, _, _, _, = ellipj(x, k ** 2)
    return result

def cn(x, k):
    _, result, _, _ = ellipj(x, k ** 2)
    return result

def dn(x, k):
    _, _, result, _ = ellipj(x, k ** 2)
    return result

def k_complete(k):
    return ellipk(k ** 2)

In [ ]:
from numpy import sqrt, pi


def first_border(k):
    theta = (-pi / 2) / sqrt(2 * (k ** 2) - 1)
    u = -(sqrt(2) / sqrt(2 * (k ** 2) - 1)) * (dn(theta, k) / sn(theta, k))
    du = (sqrt(2) / (2 * (k ** 2) - 1)) * (cn(theta, k) / (sn(theta, k) ** 2))
    return u, du

def second_border(k):
    theta = k_complete(k) - (pi / 2) / sqrt(2 * (k ** 2) + 2)   
    u = ((1 - k ** 2) / (sqrt(k ** 2 + 1))) * (sn(theta, k) / (cn(theta, k) * dn(theta, k)))
    du = (sqrt(2) / (2 * (k ** 2) + 2)) * (
        ((k ** 2) * (sn(theta, k) ** 4) - 1) * (k ** 2 - 1) / ((cn(theta, k) ** 2) * (dn(theta, k) ** 2)))
    return u, du


def third_border(k):
    theta = k_complete(k) - (pi / 2) / sqrt(2 - 4 * (k ** 2))
    u = (1 / sqrt(1 - 2 * (k ** 2))) * (sn(theta, k) * dn(theta, k) / cn(theta, k))
    du = (sqrt(2) / (2 - 4 * (k ** 2))) * (
        ((k ** 2) * (sn(theta, k) ** 4) - 2 * (k ** 2) * (sn(theta, k) ** 2) + 1) / (cn(theta, k) ** 2))
    return u, du

In [ ]:
from scipy.integrate import ode

# Notation:
#    T_* -- repultion map, -u^3, singular;
#    T_0 -- rotation map, +u^3, non-singular.

def f_repultion(x, u):
    return [u[1], u[0] + u[0] ** 3]

def f_rotation(x, u):
    return [u[1], u[0] - u[0] ** 3]

# Ode solver terminate condition.
def solout(x, u):
    if abs(u[0]) > 1e6:
        return -1
    return 1

def map_f(f, u0, du0, scale_factor=1):
    if scale_factor == 0:
        return u0, du0
    solver = ode(f).set_integrator('dopri5', method='adams')
    solver.set_initial_value((u0, du0), 0)
    solver.set_solout(solout=solout)
    u, du = solver.integrate(scale_factor * pi / 2)
    if solver.get_return_code() == 2: # terminated by solout
        return None
    return u, du

def map_repultion(u0, du0, scale_factor=1):
    return map_f(f_repultion, u0, du0, scale_factor)

def map_rotation(u0, du0, scale_factor=1):
    return map_f(f_rotation, u0, du0, scale_factor)

In [ ]:
from numpy import linspace
import matplotlib.pyplot as plt

# Compute \mathcal{U}^+ borderlines. Scheme: (3) -> (2) -> (1)
# Use epsilon to avoid border states and function degenerations
eps = 1e-5

k3 = linspace(1 / sqrt(2) - eps, 0 + eps, 500)
k2 = linspace(0 + eps, 1 - eps, 1000)
k1 = linspace(1 - eps, 1 / sqrt(2) + eps, 500)

u_span = (0, 3)
du_span = (-6, 6)

u = []
du = []

for k in k3:
    u3_current, du3_current = third_border(k)
    if abs(u3_current) < u_span[1] and abs(du3_current) < du_span[1]:
        u.append(u3_current)
        du.append(du3_current)

for k in k2:
    u2_current, du2_current = second_border(k)
    if abs(u2_current) < u_span[1] and abs(du2_current) < du_span[1]:
        u.append(u2_current)
        du.append(du2_current)

for k in k1:
    u1_current, du1_current = first_border(k)
    if abs(u1_current) < u_span[1] and abs(du1_current) < du_span[1]:
        u.append(u1_current)
        du.append(du1_current)

In [ ]:
from matplotlib.widgets import Slider


fig, ax = plt.subplots()

# Plot \mathcal{U}^+ set along with a part of the spiral -- mapping of the \mathcal{U}^+ set.
plt.plot(u, du, linewidth=1, color='black')
plt.plot([-x for x in u[::-1]], [-x for x in du[::-1]], linewidth=1, color='black')

# Points of the T_* \cdot \mathcal{U}^+
# Symmetric with respect to the du axis
u_upper = [-x for x in u]
du_upper = du

u_lower = [ x for x in u[::-1]]
du_lower = [-x for x in du[::-1]]

plt.plot(u_upper, du_upper, '--', linewidth=0.5, color='grey')
plt.plot(u_lower, du_lower, '--', linewidth=0.5, color='grey')

# Map with rotation mapping
def map_border(u, du, scale_factor):
    mapped_border = [map_rotation(u0, du0, scale_factor) for u0, du0 in zip(u, du)]
    return [_[0] for _ in mapped_border], [_[1] for _ in mapped_border]

initial_scale_factor = 0
u_upper_map, du_upper_map = map_border(u_upper, du_upper, initial_scale_factor)
u_lower_map, du_lower_map = map_border(u_lower, du_lower, initial_scale_factor)

upper_line_obj, = plt.plot(u_upper_map, du_upper_map, '--', linewidth=1, color='black') # `,` sign important
lower_line_obj, = plt.plot(u_lower_map, du_lower_map, '--', linewidth=1, color='black') # `,` sign important

# Create a slider for scale factor in mapper functions.
plt.subplots_adjust(left=0.25, bottom=0.25)
axis_scale_factor = plt.axes([0.25, 0.1, 0.65, 0.03])
flexure_slider = Slider(axis_scale_factor, 'Flexure', 0.0, 1.0, valinit=initial_scale_factor)

def update(val):
    scale_factor = flexure_slider.val
    u_upper_map, du_upper_map = map_border(u_upper, du_upper, scale_factor)
    u_lower_map, du_lower_map = map_border(u_lower, du_lower, scale_factor)
    
    upper_line_obj.set_xdata(u_upper_map)
    upper_line_obj.set_ydata(du_upper_map)
    lower_line_obj.set_xdata(u_lower_map)
    lower_line_obj.set_ydata(du_lower_map)
    
    fig.canvas.draw_idle()

# Attach `update` function to the slider
flexure_slider.on_changed(update)

plt.show()